In [ ]:
import os
# OPENAI_API_KEY="your_openai_api_key_here"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ Imports and setup complete")

✅ Imports and setup complete


In [2]:
# Tools
from langchain_core.tools import tool
DB = list()

@tool
def query_tool(query:str) -> str:
    """
    Execute a query on the database.

    Args:
        query (str): The SQL query to execute.

    Returns:
        str: The string "Query Executed!"
    """
   
    DB.append(query)
    return "Query Executed!"

In [3]:
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

def create_agent():
    """Create a LangChain agent with file reading capabilities."""
    
    # Initialize the LLM
    llm = ChatOpenAI(
        model="gpt-4.1-mini",
        temperature=0,
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )
    
    # Create tools
    tools = [query_tool]
    
    # Create the prompt template 
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a helpful query assistant that can help users generate a SQL query and execute it.
        
        Read and interpret the user's request, generate a SQL query and then select the appropriate tools to use.
        Always be helpful and follow the user's instructions carefully.

        The schema:
        TABLE grocery_items (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(100) NOT NULL,
            price DECIMAL(10, 2) NOT NULL
        );
        """),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # Create the agent
    agent = create_openai_functions_agent(llm, tools, prompt)
    
    # Create the agent executor
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        return_intermediate_steps=False
    )
    
    return agent_executor

print("✅ Agent creation function defined")

✅ Agent creation function defined


In [4]:
print("=" * 60)
print("SCENARIO 1:")
print("=" * 60)

agent = create_agent()
user_input = "Please me to add to the grocery items the following products: Cheese with a price of 80, Jam with a price of 50"

print(f"USER INPUT: {user_input}")
print("-" * 40)

try:
    result = agent.invoke({"input": user_input})
    print(f"AGENT RESPONSE: {result['output']}")
except Exception as e:
    print(f"ERROR: {str(e)}")

SCENARIO 1:
USER INPUT: Please me to add to the grocery items the following products: Cheese with a price of 80, Jam with a price of 50
----------------------------------------


> Entering new AgentExecutor chain...

Invoking: `query_tool` with `{'query': "INSERT INTO grocery_items (name, price) VALUES ('Cheese', 80), ('Jam', 50);"}`


Query Executed!The products Cheese with a price of 80 and Jam with a price of 50 have been added to the grocery items. Is there anything else you would like to do?

> Finished chain.
AGENT RESPONSE: The products Cheese with a price of 80 and Jam with a price of 50 have been added to the grocery items. Is there anything else you would like to do?
